# Tutorial : Train and deploy a rasa chatbot

The goal of this tutorial is to train and deploy a simple rasa chatbot. To do this, once you have clone the repo git, you can directly go in the folder rasa_bot and then we will train the model to have a model saved in the folder models. 

## Install the requirements

The project was created in a venv environment. To add it in a jupyter environment, you must install pip and the file requirements_rasa.txt. Run this for simplification. You just have to run the next cell one time. It permits to install rasa and play with the chatbot. You can stop and restart your notebook, the ressource will always be here. 

In [1]:
%conda install pip
%cd rasa_bot/
%pip install --no-cache-dir -r requirements_rasa.txt

Solving environment: done

## Package Plan ##

  environment location: /workspace/.miniconda3

  added / updated specs:
    - pip


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.10.11 |       h06a4308_0         124 KB
    certifi-2022.9.24          |   py39h06a4308_0         154 KB
    conda-22.9.0               |   py39h06a4308_0         884 KB
    openssl-1.1.1q             |       h7f8727e_0         2.5 MB
    pip-22.2.2                 |   py39h06a4308_0         2.3 MB
    toolz-0.12.0               |   py39h06a4308_0         105 KB
    ------------------------------------------------------------
                                           Total:         6.1 MB

The following NEW packages will be INSTALLED:

  toolz              pkgs/main/linux-64::toolz-0.12.0-py39h06a4308_0

The following packages will be UPDATED:

  ca-certificates                       2021.

## Train your model

To train the model, you have two choices. One is to use the AI Training tool, developped by OVHcloud and present in the OVHcloud control pannel. The other is to run the command here on the notebook, it will be save in a folder called models. 

### Use the AI Training tool (Recommended)

To use this tool, you can create a user account with the role AI Operator. Once this user is created, generate a token from him and save it. Create the docker image and push it to your personnal hub account. Here is the command :

```bash
docker build .  -t <yourdockerhubId>/big_chatbot:latest

docker push <yourdockerhubId>/big_chatbot:latest
```

Once this docker image is created and pushed, you can run your job. 

Create two object storage, one is for the data to read and the other is for the model to save. Is is really advice to use two object storage distinctly. 

Paste in a terminal : 

```bash
ovhai job run vvitcheff/big_chatbot:latest \ 

--gpu 2 \ 

--volume <containerwithdata>@<region>/:/workspace/data:RO \ 

--volume <containerforsavingthemodel>@<region>/rasa-models:/workspace/rasa_bot/models:RW \ 

-- bash -c "cd rasa_bot && rasa train --force --fixed-model-name customer-model" 
```

The training of the model is about 4 minutes. Your model will be save in the container <containerforsavingthemodel> mounted in the folder rasa-models. The name of the model will be "customer-model.tar.gz". You can synchronize this model to have it on your notebook. Here is the command to do so :

```bash
ovhai notebook pull-data <notebookid>
```
    
Once this is done, you have train your chatbot and he is ready to be use. 
    
### Train your model here 
    
To train your model here, simply run the next cell. When your model will be save, don't forget to copy and paste it into the folder saved_models. 

In [5]:
import nest_asyncio
import rasa

nest_asyncio.apply()
print("Event loop ready.")

#Show all of the directory
print(os.listdir("."))

#Create the variable of the direction of the files
config = "config.yml"
training_files = "data/"
domain = "domain.yml"
output = "models/"
print(config, training_files, domain, output)

#Train the model 
model_path = rasa.train(domain, config, [training_files], output)
print(model_path)
#The store variable will be use after to test the chatbot. 

Event loop ready.
['.dockerignore', 'rasa.Dockerfile', 'tests', 'actions', 'domain.yml', 'endpoints.yml', 'requirements_rasa.txt', 'credentials.yml', '.rasa', 'models', 'config.yml', 'data']
config.yml data/ domain.yml models/


/workspace/.miniconda3/lib/python3.9/site-packages/rasa/shared/core/slot_mappings.py:221: UserWarning: Slot auto-fill has been removed in 3.0 and replaced with a new explicit mechanism to set slots. Please refer to https://rasa.com/docs/rasa/domain#slots to learn more.
  rasa.shared.utils.io.raise_warning(
Processed rules: 100%|██████████| 8/8 [00:00<00:00, 1028.30it/s, # trackers=1]
Processed trackers: 0it [00:00, ?it/s]
Processed trackers: 100%|██████████| 8/8 [00:00<00:00, 647.13it/s, # action=19]
Processed actions: 19it [00:00, 1637.73it/s, # examples=17]
Processed trackers: 0it [00:00, ?it/s]
Processed trackers: 100%|██████████| 8/8 [00:00<00:00, 1289.96it/s]


Your Rasa model is trained and saved at 'models/20221103-093032-taxonomic-poltergeist.tar.gz'.
TrainingResult(model='models/20221103-093032-taxonomic-poltergeist.tar.gz', code=0, dry_run_results=None)


## Play with the chatbot

Now that we have our model, let's try to play with the chatbot and speak with him. Run the above cell and you will be able to play with him. You can run in a terminal rasa shell to chat with the bot. 

## Deploy the rasa API

Once you're finish playing with the chatbot, it could be interesting to deploy it with the tool AI Deploy from rasa. To do so, please create new docker image and push it to your repository dockerhub. Here are the two commands to do (don't forget to go at the root of the directory rasa_chatbot :

```bash
docker build . -f deploy.Dockerfile -t <yourdockerhubId>/rasa-model:latest

docker push <yourdockerhubId>/rasa-model:latest
```

Now, with the ovhai CLI, you can run with a simple line the api of rasa where we will be able to do some post and get requests on it. Here is the simple command to run :

```bash
ovhai app run –name rasa-model \

–token <token> \

–default-http-port 5005 \

–cpu 4 \

<yourdockerhubId>/rasa-model:latest
```

Once your app is started, you can do some post requests and get requests with the tool curl directly on the terminal. 